In [ ]:
SIGNAL = "illuminance"
GAP_THRESHOLD = "2min"
DAYS=30

# Valid reading limits
LOWER_LIMIT = 0
UPPER_LIMIT = 15100

# Noise detection rolling window size
NOISE_WINDOW = 10

In [ ]:
%run ../pathutils.ipynb
%run ../database.ipynb
%run ../export.ipynb
%run health.ipynb
%run database.ipynb
%run utils.ipynb

In [ ]:
import pandas as pd

# Load the readings
df = load_sensor_readings("veml7700", DAYS)

# Parse timestamp as timezone-aware UTC, set it as the index and sort
df['timestamp'] = pd.to_datetime(df['timestamp'], utc=True)
df = df.set_index('timestamp').sort_index()

# Sensor Health Analysis

In [ ]:
# Analyse sensor health
df, gaps, oor_points = analyse_health(df, SIGNAL, GAP_THRESHOLD, LOWER_LIMIT, UPPER_LIMIT, NOISE_WINDOW)

In [ ]:
# Get the export folder path
export_folder_path = get_export_folder_path("health")

# Data Gaps

In [ ]:
chart_data_gaps(df, SIGNAL, gaps, export_folder_path, f"veml7700_{SIGNAL}_gaps")

# Out-of-Range Data

In [ ]:
chart_out_of_range(df, SIGNAL, oor_points, LOWER_LIMIT, UPPER_LIMIT, export_folder_path, f"veml7700_{SIGNAL}_out_of_range")

# Sensor Noise

In [ ]:
chart_sensor_noise(df, SIGNAL, export_folder_path, f"veml7700_{SIGNAL}_noise")

# Saturation Timeline

In [ ]:
plt.figure(figsize=(14, 2))
plt.plot(df.index, df['issaturated'].astype(int), drawstyle='steps-post')
plt.title("VEML7700 Saturation Timeline")

# Export to PNG or PDF, if required
export_chart(export_folder_path, f"veml7700_{SIGNAL}_saturation", "png")

plt.show()